In [ ]:
#81
import torch
import torch.nn as nn

vocab_size = 4
emb_dim = 5
embeddings = nn.Embedding(vocab_size, emb_dim)
# 0番目の単語なので，[0]をTensorに変換
word = torch.tensor([0,1,2,3])
embed_word = embeddings(word)
print(embed_word)
print(word.shape, '->', embed_word.shape)

tensor([[ 0.7597, -0.2045, -0.2577,  0.2245, -1.1226],
        [ 1.0200, -1.2710,  0.0566, -1.2004, -0.1507],
        [ 0.5847,  0.8397, -1.1634, -0.7765,  0.3819],
        [-0.2148,  0.7816, -0.2768,  1.5894,  0.6335]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([4]) -> torch.Size([4, 5])


In [ ]:
import torch
from torch import nn

class RNN(nn.Module):
  def __init__(self, vocab_size, emb_size, padding_idx, output_size, hidden_size):
    super().__init__()
    self.hidden_size = hidden_size
    self.emb = nn.Embedding(vocab_size, emb_size, padding_idx=padding_idx)
    self.rnn = nn.RNN(emb_size, hidden_size, nonlinearity='tanh', batch_first=True)
    self.fc = nn.Linear(hidden_size, output_size)

  def forward(self, x):
    self.batch_size = x.size()[0]
    hidden = self.init_hidden(x.device)  # h0のゼロベクトルを作成
    emb = self.emb(x)
    # emb.size() = (batch_size, seq_len, emb_size)
    out, hidden = self.rnn(emb, hidden)
    # out.size() = (batch_size, seq_len, hidden_size)
    out = self.fc(out[:, -1, :])
    # out.size() = (batch_size, output_size)
    return out

  def init_hidden(self, device):
    hidden = torch.zeros(1, self.batch_size, self.hidden_size, device=device)
    return hidden

In [ ]:
from torch.utils.data import Dataset

class CreateDataset(Dataset):
  def __init__(self, X, y, tokenizer):
    self.X = X
    self.y = y
    self.tokenizer = tokenizer

  def __len__(self):  # len(Dataset)で返す値を指定
    return len(self.y)

  def __getitem__(self, index):  # Dataset[index]で返す値を指定
    text = self.X.iloc[index]
    inputs = self.tokenizer(text)

    return ({'inputs': torch.tensor(inputs, dtype=torch.int64),'labels': torch.tensor(self.y[index], dtype=torch.int64)})

In [ ]:
# ラベルベクトルの作成
category_dict = {'b': 0, 't': 1, 'e':2, 'm':3}
y_train = train['CATEGORY'].map(lambda x: category_dict[x]).values
y_valid = valid['CATEGORY'].map(lambda x: category_dict[x]).values
y_test = test['CATEGORY'].map(lambda x: category_dict[x]).values

# Datasetの作成
dataset_train = CreateDataset(train['TITLE'], y_train, tokenizer)
dataset_valid = CreateDataset(valid['TITLE'], y_valid, tokenizer)
dataset_test = CreateDataset(test['TITLE'], y_test, tokenizer)

print(f'len(Dataset)の出力: {len(dataset_train)}')
print('Dataset[index]の出力:')
for var in dataset_train[1]:
  print(f'  {var}: {dataset_train[1][var]}')

len(Dataset)の出力: 10672
Dataset[index]の出力:
  inputs: tensor([  55,   59,  161,    4,    7,  234, 3530,   26,   97,  429])
  labels: 0


In [ ]:
import torch
x = torch.zeros(2, 3, 4, 5, 6)
print(x.size())
print(x.unsqueeze(1).size())
print(x.size())
print(x.size()[0:2])

torch.Size([2, 3, 4, 5, 6])
torch.Size([2, 1, 3, 4, 5, 6])
torch.Size([2, 3, 4, 5, 6])
torch.Size([2, 3])


In [ ]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
print(input)
print("\n")
print(output)

tensor([[-1.3049,  1.0181,  0.4924],
        [-0.9001,  1.3291, -0.4746]])


tensor([[0.0580, 0.5920, 0.3500],
        [0.0846, 0.7860, 0.1294]])


In [ ]:
# パラメータの設定
VOCAB_SIZE = len(set(word2id.values())) + 1  # 辞書のID数 + パディングID
EMB_SIZE = 300
PADDING_IDX = len(set(word2id.values()))
OUTPUT_SIZE = 4
HIDDEN_SIZE = 50

# モデルの定義
model = RNN(VOCAB_SIZE, EMB_SIZE, PADDING_IDX, OUTPUT_SIZE, HIDDEN_SIZE)

# 先頭10件の予測値取得
for i in range(10):
  X = dataset_train[i]['inputs'] #titleのID表記したtensor
  print(torch.softmax(model(X.unsqueeze(0)), dim=-1)) #unsqueezeでバッチの大きさ1を追加

tensor([[0.2413, 0.3772, 0.2529, 0.1285]], grad_fn=<SoftmaxBackward0>)
tensor([[0.1302, 0.2119, 0.2848, 0.3732]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2102, 0.3164, 0.1220, 0.3514]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2192, 0.3151, 0.2845, 0.1812]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3727, 0.3245, 0.1227, 0.1800]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3112, 0.3226, 0.2402, 0.1261]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2120, 0.2444, 0.2365, 0.3071]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2857, 0.2372, 0.3125, 0.1646]], grad_fn=<SoftmaxBackward0>)
tensor([[0.2719, 0.3048, 0.2785, 0.1449]], grad_fn=<SoftmaxBackward0>)
tensor([[0.3448, 0.3059, 0.1831, 0.1661]], grad_fn=<SoftmaxBackward0>)
